In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import optbinning as opt
%matplotlib inline
import pandas as pd
from sklearn.impute import SimpleImputer
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.preprocessing import OneHotEncoder, StandardScaler,KBinsDiscretizer, LabelEncoder
from sklearn.impute import SimpleImputer
from scipy.stats import mannwhitneyu ,chi2_contingency, anderson, f_oneway
import statsmodels.api as sm 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
left_data=pd.read_csv("../data/application_train_vf.csv",parse_dates=["date_mensuelle"], index_col=0)

In [ ]:
data=pd.read_csv("../data/bureau.csv") #

In [ ]:
left_data=left_data[["SK_ID_CURR","TARGET"]]

In [ ]:
data

In [ ]:
data.isna().mean()

In [ ]:
data["CREDIT_ACTIVE"].value_counts()

In [ ]:
(data["AMT_CREDIT_SUM_DEBT"] > 0).mean()

In [ ]:
data["CREDIT_TYPE"].value_counts()

In [ ]:
data["CREDIT_ACTIVE"].value_counts()

In [ ]:
sns.kdeplot(data[data["CREDIT_ACTIVE"]=="Active"]["AMT_ANNUITY"].apply(np.log))
plt.show()

In [ ]:
(data["AMT_CREDIT_MAX_OVERDUE"] == 0 ).mean()

In [ ]:
(data["CREDIT_DAY_OVERDUE"] >= 30).mean()

In [ ]:
data_credit_bureau=left_data.merge(data, how='left', on="SK_ID_CURR")

In [ ]:
data_credit_bureau.isna().mean()

In [ ]:
data_credit_bureau[["AMT_CREDIT_SUM_DEBT","AMT_CREDIT_SUM","AMT_ANNUITY"]].fillna(0, inplace=True)

In [ ]:
clients_debts=data_credit_bureau[["SK_ID_CURR", "AMT_CREDIT_SUM_DEBT"]].groupby(by="SK_ID_CURR").agg(sum).reset_index()

In [ ]:
clients_all_credits=data_credit_bureau.loc[data_credit_bureau["CREDIT_ACTIVE"]=="Active",["SK_ID_CURR","AMT_CREDIT_SUM"]].groupby(by="SK_ID_CURR").sum().reset_index()

In [ ]:
clients_all_annuities=data_credit_bureau.loc[data_credit_bureau["CREDIT_ACTIVE"]=="Active",["SK_ID_CURR","AMT_ANNUITY"]].groupby(by="SK_ID_CURR").sum().reset_index()

In [ ]:
clients_credit_actives=data_credit_bureau.loc[data_credit_bureau["CREDIT_ACTIVE"]=="Active",["SK_ID_CURR","CREDIT_ACTIVE"]].groupby(by="SK_ID_CURR").count().reset_index()

In [ ]:
clients_credit_closed=data_credit_bureau.loc[data_credit_bureau["CREDIT_ACTIVE"]=="Closed",["SK_ID_CURR","CREDIT_ACTIVE"]].groupby(by="SK_ID_CURR").count().reset_index()

In [ ]:
last_credit=data_credit_bureau.loc[data_credit_bureau["CREDIT_ACTIVE"]=="Active",["SK_ID_CURR","DAYS_CREDIT"]].groupby(by="SK_ID_CURR").max().reset_index()

In [ ]:
end_date_last_credit=data_credit_bureau.loc[data_credit_bureau["CREDIT_ACTIVE"]=="Active",["SK_ID_CURR","DAYS_CREDIT_ENDDATE"]].groupby(by="SK_ID_CURR").max().reset_index()

In [ ]:
clients_credit_actives.rename(columns={"CREDIT_ACTIVE":"NB_CREDIT_ACTIVE"}, inplace=True)
clients_credit_closed.rename(columns={"CREDIT_ACTIVE":"NB_CREDIT_CLOSED"}, inplace=True)

In [ ]:
left_data=left_data.merge(clients_debts, how="left", on = "SK_ID_CURR")

In [ ]:
left_data=left_data.merge(clients_credit_actives, how="left", on = "SK_ID_CURR")

In [ ]:
left_data=left_data.merge(clients_credit_closed, how="left", on = "SK_ID_CURR")

In [ ]:
left_data=left_data.merge(last_credit, how="left", on ="SK_ID_CURR")

In [ ]:
left_data=left_data.merge(end_date_last_credit, how="left", on = "SK_ID_CURR")

In [ ]:
left_data=left_data.merge(clients_all_credits, how="left", on="SK_ID_CURR")

In [ ]:
left_data=left_data.merge(clients_all_annuities, how="left", on="SK_ID_CURR")

In [ ]:
left_data["DAYS_CREDIT"]

In [ ]:
left_data["NB_CREDIT_ACTIVE"].fillna(0, inplace=True)
left_data["NB_CREDIT_CLOSED"].fillna(0, inplace=True)
left_data["DAYS_CREDIT"].fillna(left_data["DAYS_CREDIT"].max(), inplace=True)
left_data["AMT_CREDIT_SUM_DEBT"].fillna(0, inplace=True)
left_data["AMT_CREDIT_SUM"].fillna(0, inplace=True)

In [ ]:
left_data["DAYS_CREDIT_ENDDATE"].fillna(0, inplace=True)

In [ ]:
left_data["AMT_ANNUITY"].fillna(0, inplace=True)

In [ ]:
left_data.drop(columns="TARGET").add_prefix("CB_").to_csv("../data/cb_findings.csv")

In [ ]:
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

In [ ]:
def mannwhitney_test(df:pd.DataFrame,variable:str,target:str):
    # Séparer les données en deux groupes en fonction de la variable cible
    group_1 = df[df[target] == 0]
    group_2 = df[df[target] == 1]

    stat, p_value = mannwhitneyu(group_1[variable].dropna(), group_2[variable].dropna())
    # Afficher les résultats
    print(f"Variable: {variable}")
    print(f"Mann-Whitney U-statistic: {stat}")
    print(f"P-value: {p_value}")
    print(10*"===")
    
    pass

In [ ]:
mannwhitney_test(left_data,"AMT_CREDIT_SUM","TARGET")

In [ ]:
mannwhitney_test(left_data,"AMT_CREDIT_SUM_DEBT","TARGET")

In [ ]:
mannwhitney_test(left_data,"NB_CREDIT_CLOSED","TARGET")

In [ ]:
mannwhitney_test(left_data,"NB_CREDIT_ACTIVE","TARGET")

In [ ]:
mannwhitney_test(left_data,"DAYS_CREDIT","TARGET")

In [ ]:
mannwhitney_test(left_data,"DAYS_CREDIT_ENDDATE","TARGET")

In [ ]:
mannwhitney_test(left_data,"AMT_ANNUITY","TARGET")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

class DecisionTreeDiscretizer:
    def __init__(self, max_bins=5, target=None):
        self.tree_max_bins = max_bins
        self.clf = DecisionTreeClassifier(criterion="gini", max_depth=int(round(self.tree_max_bins/2,0)),
                                          min_samples_split=0.05,
                                           min_samples_leaf=0.05) #
        self.target = target

    def fit(self, X_train):
        # Entraîner le modèle sur les données d'entraînement
        self.clf.fit(X_train.values.reshape(-1, 1), self.target)

    def _get_tree_thresholds(self):
        thresholds = set()
        for node in range(self.clf.tree_.node_count):
            if self.clf.tree_.children_left[node] != self.clf.tree_.children_right[node]:  # non-leaf node
                feature = self.clf.tree_.feature[node]
                threshold = self.clf.tree_.threshold[node]
                thresholds.add(threshold)
        return np.array(list(thresholds))

    def get_thresholds(self):
        if hasattr(self.clf, 'tree_') and self.clf.tree_ is not None:
            thresholds_np = self._get_tree_thresholds()
            thresholds = [-np.inf] + sorted(list(set(list(thresholds_np)))) + [np.inf]
            return thresholds
        else:
            raise ValueError("Le classifieur n'est pas entraîné. Utilisez la méthode fit avant d'obtenir les seuils.")

    def transform(self, X):
        # Vérifier si le modèle est entraîné
        if not hasattr(self.clf, 'tree_') or self.clf.tree_ is None:
            raise ValueError("Le classifieur n'est pas entraîné. Utilisez la méthode fit avant de transformer les données de test.")

        thresholds = self.get_thresholds()

        # Utiliser pd.cut pour obtenir les intervalles au lieu des numéros
        intervals = pd.cut(X.values.flatten(), bins=thresholds, include_lowest=True, right=True)
        return intervals



In [ ]:
discretizer=DecisionTreeDiscretizer(target=left_data["TARGET"])

In [ ]:
discretizer.fit(left_data["AMT_CREDIT_SUM_DEBT"])
left_data["AMT_CREDIT_SUM_DEBT"]=discretizer.transform(left_data["AMT_CREDIT_SUM_DEBT"])

In [ ]:
discretizer.fit(left_data["NB_CREDIT_CLOSED"])
left_data["NB_CREDIT_CLOSED"]=discretizer.transform(left_data["NB_CREDIT_CLOSED"])

In [ ]:
discretizer.fit(left_data["NB_CREDIT_ACTIVE"])
left_data["NB_CREDIT_ACTIVE"]=discretizer.transform(left_data["NB_CREDIT_ACTIVE"])

In [ ]:
discretizer.fit(left_data["DAYS_CREDIT"])
left_data["DAYS_CREDIT"]=discretizer.transform(left_data["DAYS_CREDIT"])

In [ ]:
discretizer.fit(left_data["DAYS_CREDIT_ENDDATE"])
left_data["DAYS_CREDIT_ENDDATE"]=discretizer.transform(left_data["DAYS_CREDIT_ENDDATE"])

In [ ]:
discretizer.fit(left_data["AMT_CREDIT_SUM"])
left_data["AMT_CREDIT_SUM"]=discretizer.transform(left_data["AMT_CREDIT_SUM"])

In [ ]:
discretizer.fit(left_data["AMT_ANNUITY"])
left_data["AMT_ANNUITY"]=discretizer.transform(left_data["AMT_ANNUITY"])

In [ ]:
pd.crosstab(left_data["TARGET"], left_data["NB_CREDIT_ACTIVE"])

In [ ]:
def calculate_information_value_from_contingency_table(contingency_table):
    """
    Calculer l'Information Value à partir d'une table de contingence.
    
    Args:
        contingency_table (pd.DataFrame): La table de contingence entre la variable et la target.
        
    Returns:
        float: L'Information Value calculé.
    """
    non_event_rate=contingency_table.iloc[0]/(contingency_table.iloc[0].sum())
    event_rate=contingency_table.iloc[1]/(contingency_table.iloc[1].sum())
    iv=0
    if (non_event_rate.min() > 0) & (event_rate.mean() >0) :
        for col in non_event_rate.index:
            iv += (event_rate[col] - non_event_rate[col])*np.log(event_rate[col] / non_event_rate[col])

    return iv

In [ ]:
calculate_information_value_from_contingency_table(pd.crosstab(left_data["TARGET"], left_data["AMT_CREDIT_SUM_DEBT"]))

In [ ]:
calculate_information_value_from_contingency_table(pd.crosstab(left_data["TARGET"], left_data["NB_CREDIT_ACTIVE"]))

In [ ]:
calculate_information_value_from_contingency_table(pd.crosstab(left_data["TARGET"], left_data["NB_CREDIT_CLOSED"]))

In [ ]:
calculate_information_value_from_contingency_table(pd.crosstab(left_data["TARGET"], left_data["DAYS_CREDIT"]))

In [ ]:
calculate_information_value_from_contingency_table(pd.crosstab(left_data["TARGET"], left_data["DAYS_CREDIT_ENDDATE"]))

In [ ]:
calculate_information_value_from_contingency_table(pd.crosstab(left_data["TARGET"], left_data["AMT_CREDIT_SUM"]))

In [ ]:
calculate_information_value_from_contingency_table(pd.crosstab(left_data["TARGET"], left_data["AMT_ANNUITY"]))